Import stuff

In [15]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np

#pv.set_jupyter_backend("ipyvtklink")

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

Import Envelope Lattice

In [16]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/meshes/voxelized_envelope.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
envelope_lattice.shape

env_cens = envelope_lattice.centroids_threshold(-1)

print (env_cens)

[[ -40. -100.    0.]
 [ -40. -100.    5.]
 [ -40. -100.   10.]
 ...
 [ 110.  -15.   25.]
 [ 110.  -15.   30.]
 [ 110.  -15.   35.]]


Calculate the border points (this calculates the border voxels of the envelope, not the points on the street)

In [17]:
border_lattice = envelope_lattice * 0.0
# border_lattice[:,:,1:-1] += 1.0
border_lattice[:,1:-1,:] += 1.0
border_lattice[1:-1,:,:] += 1.0
border_lattice = border_lattice != 2
border_lattice.centroids

cloud([[ -40., -100.,    0.],
       [ -40., -100.,    5.],
       [ -40., -100.,   10.],
       ...,
       [ 110.,  -15.,   25.],
       [ 110.,  -15.,   30.],
       [ 110.,  -15.,   35.]])

Plotting the code above

In [18]:
p = pv.Plotter(notebook=True)

base_lattice = border_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
envelope_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 1.1])
# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)


# p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(239.8279062084888, 147.3279062084888, 222.3279062084888),
 (35.0, -57.5, 17.5),
 (0.0, 0.0, 1.0)]

Calculating the points on the street (defect)

In [19]:
#calculating maximum and minimum values

#minimum values for x and y
xmin, ymin, zmin = env_cens.min(axis=0)

#maximum values for x and y
xmax, ymax, zmax = env_cens.max(axis=0)

#change bound
xmin = xmin - 10
ymin = ymin - 10
xmax = xmax + 10
ymax = ymax + 10

#calculating the points on the street

#calculate the x coördinates
x = np.arange(xmin, xmax)

#calculate the y coördinates
y = np.arange(ymin, ymax)

X = np.tile(x, (2, 1))
X = np.append(X, np.tile(xmin, (len(y), 1)))
X = np.append(X, np.tile(xmax, (len(y), 1)))

Y = np.tile(ymin, (len(x), 1))
Y = np.append(Y, np.tile(ymax, (len(x), 1)))
Y = np.append(Y, np.tile(y, (2, 1)))

Z = np.zeros(len(X))

street_points = np.stack([X,Y,Z]).T
street_points = tg.cloud(street_points)


Visualizing

In [20]:
p = pv.Plotter(notebook=True)

street_points.fast_notebook_vis(p)

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
envelope_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")


# Add the data values to the cell data
grid.cell_arrays["Agents"] = base_lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([0.9, 1.1])
# adding the voxels
#p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)


# p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(262.28353055475134, 169.7835305547513, 244.7835305547513),
 (35.0, -57.5, 17.5),
 (0.0, 0.0, 1.0)]